<a href="https://colab.research.google.com/github/s1250103/Public_mori-lab/blob/eras/eras/tune/programs/modelTuner_Drop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setting

In [1]:
%tensorflow_version 2.x

In [2]:
!pip install -q keras_tuner

In [4]:
def beep():
  from google.colab import output
  output.eval_js('new Audio(\
"https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg")\
.play()') 


import

In [6]:
# file dealing
import os
import shutil
from google.colab import drive
# data dealing
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
# process deasing
import gc

# machine learning (back)
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras import layers, models, initializers, callbacks

# machine learning
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils.np_utils import to_categorical

seed = 20201218
np.random.seed(seed)
tf.random.set_seed(seed)

データフォーマットを規定

In [7]:
class video_format:
  name = "video_format"
  # サンプリングされたCMデータの仕様
  playtime = "15秒"
  displaysize = "(any, any, RGB)"
  videoformat = "any"
  # モデルが扱うCMデータ(上のようなデータは、下のように変換される)
  HEIGHT = 45
  WIDTH = 80
  FRAME_SIZE = 30
  COLORinfo = 3 # "RGB"
  FPS = "2 (FRAME_SIZE / playtime)" # 定義ではなく上から導かれた値

format1 = video_format()

データをインポート

In [5]:
#@title gdrive マウント
if not os.path.exists('/content/drive'):
  drive.mount('/content/drive')
else:
  print("Already confirm")


Mounted at /content/drive


In [6]:
#@title 一時的にファイルを置くディレクトリ'desk'を作成
desk = '/content/desk'
if not os.path.exists(desk):
  os.mkdir(desk)
os.chdir(desk)

In [7]:
#@title 必要なファイルを'desk'にコピー
%%time
wants_paths = [
'/content/drive/MyDrive/colab/dence1223/normalTrainLabels.npz',
'/content/drive/MyDrive/colab/dence1223/normalTrainVideos.npz',
'/content/drive/MyDrive/colab/dence1223/normalTestLabels.npz',
'/content/drive/MyDrive/colab/dence1223/normalTestVideos.npz'
]

for want in wants_paths:
  if not os.path.exists(os.path.join(desk, os.path.basename(want))):
    shutil.copy2(want, desk)
    print("get : ", want)
            

get :  /content/drive/MyDrive/colab/dence1223/normalTrainLabels.npz
get :  /content/drive/MyDrive/colab/dence1223/normalTrainVideos.npz
get :  /content/drive/MyDrive/colab/dence1223/normalTestLabels.npz
get :  /content/drive/MyDrive/colab/dence1223/normalTestVideos.npz
CPU times: user 75.6 ms, sys: 450 ms, total: 526 ms
Wall time: 5.36 s


# model creating and hyper prametar designing

In [29]:
from tensorflow import keras
from tensorflow.keras.layers import Dense

def build_model(hp):
    # モデル作成
    model = models.Sequential()
    # 入力層
    model.add(
        layers.Reshape(
            (format1.FRAME_SIZE,
            format1.HEIGHT,
            format1.WIDTH,
            format1.COLORinfo),
            input_shape=(format1.FRAME_SIZE * format1.HEIGHT * format1.WIDTH * format1.COLORinfo,),
            name='ENTRANCE' ))
    # 中間層
    ## dropout_0
    # model.add(layers.Dropout(0.1))
    model.add(layers.Dropout(hp.Choice('drop0', values=[1e-7, 0.3, 0.5, 0.8])))

    ## 畳み込み0
    model.add(
        layers.Conv3D(
            filters=32,
            kernel_size=(2, 3, 3),
            strides=(2, 3, 3),
            padding='same',
            activation='relu',
            name='conv0'))

    # model.add(layers.Conv3D(hp.Choice('conv0_units',
    #                                   values=[16, 32, 64]),
    #                         (2,3,3), padding='same',
    #                     kernel_initializer=initializers.TruncatedNormal(),
    #                     use_bias=True, activation='relu',
    #                     name='conv_filter'))         
                            

    ## pool0
    model.add(
        layers.MaxPooling3D(pool_size=(2, 3, 3), name='pool0'))

    # dropout_1
    model.add(layers.Dropout(0.1))
    # model.add(layers.Dropout(hp.Choice('drop1', values=[0.1, 0.3, 0.5])))
        

    ## 全結合0
    model.add(
        layers.Flatten(name='pipe'),
    )
    model.add(
        layers.Dense(1024,
          activation='relu',
          name='DAM' ), )
    # model.add(
    #         layers.Dense(units=hp.Choice('units_dence0', values=[512, 1024, 2048]),
    #                     activation='relu',
    #                     kernel_initializer=initializers.TruncatedNormal(),
    #                     name='hidden'))          

    # 出力層
    model.add(
        layers.Dense(4, activation='softmax', name='WATERSUPPLY')
    )

    # 最適化
    adam = keras.optimizers.Adam(
        lr=1e-7, beta_1=0.9, beta_2=0.999,
        epsilon=None, decay=0.0, amsgrad=False)
    # 誤差関数
    mse = tf.keras.losses.MeanSquaredError()
    cce = tf.keras.losses.CategoricalCrossentropy()

    model.compile(
        optimizer=adam,
        loss=cce,
        metrics=['acc', cce])
      
    return model

# tuner designing

In [25]:
class p_name:
  num = 'str'
  def __init__(self):
    self.num = 0
  def use(self):
    self.num = self.num + 1
    return str(self.num)

p1 = p_name()

In [31]:
from kerastuner.tuners import Hyperband


tuner = Hyperband(
    build_model,
    objective='val_acc',
    max_epochs=5,
    directory='my_dir',
    project_name=p1.use(),
    overwrite=False
    )

In [32]:
tuner.search_space_summary()

Search space summary
Default search space size: 1
drop0 (Choice)
{'default': 1e-07, 'conditions': [], 'values': [1e-07, 0.3, 0.5, 0.8], 'ordered': True}


# gathering data

In [13]:
def prepare_data():
  # traindata
  v = np.load('/content/desk/normalTrainVideos.npz')
  l = np.load('/content/desk/normalTrainLabels.npz')

  train_videos = [] 
  train_labels = []
  for i in v.files:
    train_videos.append(v[i])
  for i in l.files:
    train_labels.append(l[i])

  train_videos = np.array(train_videos)
  train_labels = np.array(train_labels)
  train_labels = tf.keras.utils.to_categorical(train_labels, 4)

  # testdata
  v = np.load('/content/desk/normalTestVideos.npz')
  l = np.load('/content/desk/normalTestLabels.npz')

  test_videos = []
  test_labels = []
  for i in v.files:
    test_videos.append(v[i])
  for i in l.files:
    test_labels.append(l[i])

  test_videos = np.array(test_videos)
  test_labels = np.array(test_labels)
  test_labels = tf.keras.utils.to_categorical(test_labels, 4)
  
  return train_videos, train_labels,  test_videos, test_labels

In [14]:
train_videos, train_labels,  test_videos, test_labels = prepare_data()

# searching param

In [33]:
%%time
tuner.search(train_videos, train_labels,
             epochs=1024,
             validation_data=(test_videos, test_labels))
beep()

Trial 4 Complete [00h 00m 01s]
val_acc: 0.3306451737880707

Best val_acc So Far: 0.3306451737880707
Total elapsed time: 00h 00m 06s
INFO:tensorflow:Oracle triggered exit
CPU times: user 6.22 s, sys: 620 ms, total: 6.84 s
Wall time: 7.13 s


# reloading tuner

In [18]:
# reload existing tuner
tuner = Hyperband(
    build_model,
    objective='val_loss',
    max_epochs=0,
    directory='my_dir',
    project_name=str(p_name),
    overwrite=False
    )

INFO:tensorflow:Reloading Oracle from existing project my_dir/5/oracle.json
INFO:tensorflow:Reloading Tuner from my_dir/5/tuner0.json


# confirming the contents

In [34]:
tuner.results_summary()

Results summary
Results in my_dir/4
Showing 10 best trials
Objective(name='val_acc', direction='max')
Trial summary
Hyperparameters:
drop0: 1e-07
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.3306451737880707
Trial summary
Hyperparameters:
drop0: 0.5
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.2822580635547638
Trial summary
Hyperparameters:
drop0: 0.3
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.14516128599643707
Trial summary
Hyperparameters:
drop0: 0.8
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.13709677755832672


In [32]:
!rm -r my_dir/